In [2]:
import pandas  as pd
import pymongo as db
import config
from bson.objectid import ObjectId
from typing import Callable

#### teste

In [ ]:
client = db.MongoClient(config.MONGO_URI)

try:
    database = client.get_database("saturnDB")
    collection = database.get_collection("metrics")    
    query = { "organization_id": ObjectId("6745d7873f4e39e161319575"),
              "year": 2023,
              "month": 1 }
    metrics = collection.find_one(query)
    client.close()
except Exception as e:
    raise Exception("Unable to retrieve the document due to the following error: ", e)

if (not metrics):
    raise Exception("Collection not found.")
else:
    try:
        df = pd.json_normalize(metrics)
    except ValueError as e:
        raise Exception("Unable to load data from the collection: ", e)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 99 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   _id                                    1 non-null      object
 1   organization_id                        1 non-null      object
 2   year                                   1 non-null      int64 
 3   month                                  1 non-null      int64 
 4   partos_vaginais                        1 non-null      int64 
 5   partos_cesareos                        1 non-null      int64 
 6   saidas_clinicas_anterior               1 non-null      int64 
 7   saidas_cirurgicas_anterior             1 non-null      int64 
 8   reinternacoes_clinicas                 1 non-null      int64 
 9   reinternacoes_cirurgicas               1 non-null      int64 
 10  pacientes_dia                          1 non-null      int64 
 11  pcr_eventos            

In [ ]:
def validar_kwargs(kwargs, chaves_obrigatorias):
    # Validar se kwargs está vazio
    if not kwargs:
        raise ValueError("Os argumentos kwargs não podem estar vazios.")
    
    # Validar se todas as chaves obrigatórias estão presentes
    for chave in chaves_obrigatorias:
        if chave not in kwargs:
            raise KeyError(f"A chave obrigatória '{chave}' está ausente.")
    
    # Validar se os valores das chaves obrigatórias não são nulos
    for chave in chaves_obrigatorias:
        if kwargs[chave] is None:
            raise ValueError(f"O valor da chave '{chave}' não pode ser nulo.")
    
    # Retornar True se tudo estiver válido
    return True

def kpi_tempo_medio(numerador,denominador):
    return (numerador/denominador)

def kpi5(**kwargs):
    # 5. Tempo médio de internação
    chaves_obrigatorias = [
        'cli_pedi_total_pacientes_dia','cli_pedi_total_saidas',
        'cli_ad_total_pacientes_dia','cli_ad_total_saidas',
        'cli_idoso_total_pacientes_dia','cli_idoso_total_saidas',
        'cir_pedi_total_pacientes_dia','cir_pedi_total_saidas',
        'cir_ad_total_pacientes_dia','cir_ad_total_saidas',
        'cir_idoso_total_pacientes_dia','cir_idoso_total_saidas'
        ]
    if (validar_kwargs(kwargs,chaves_obrigatorias)):
        # Calcular o KPI        
        cli_total_pacientes_dia = kwargs['cli_pedi_total_pacientes_dia']+kwargs['cli_ad_total_pacientes_dia']+kwargs['cli_idoso_total_pacientes_dia']
        cir_total_pacientes_dia = kwargs['cir_pedi_total_pacientes_dia']+kwargs['cir_ad_total_pacientes_dia']+kwargs['cir_idoso_total_pacientes_dia']
        pedi_total_pacientes_dia = kwargs['cir_pedi_total_pacientes_dia']+kwargs['cli_pedi_total_pacientes_dia']
        ad_total_pacientes_dia = kwargs['cir_ad_total_pacientes_dia']+kwargs['cli_ad_total_pacientes_dia']
        idoso_total_pacientes_dia = kwargs['cir_idoso_total_pacientes_dia']+kwargs['cli_idoso_total_pacientes_dia']
        cli_total_saidas = kwargs['cli_pedi_total_saidas']+kwargs['cli_ad_total_saidas']+kwargs['cli_idoso_total_saidas']
        cir_total_saidas = kwargs['cir_pedi_total_saidas']+kwargs['cir_ad_total_saidas']+kwargs['cir_idoso_total_saidas']
        pedi_total_saidas = kwargs['cir_pedi_total_saidas']+kwargs['cli_pedi_total_saidas']
        ad_total_saidas = kwargs['cir_ad_total_saidas']+kwargs['cli_ad_total_saidas']
        idoso_total_saidas = kwargs['cir_idoso_total_saidas']+kwargs['cli_idoso_total_saidas']
        total_pacientes_dia = cli_total_pacientes_dia + cir_total_pacientes_dia
        total_saidas = cli_total_saidas + cir_total_saidas

        res_cli_pedi = kpi_tempo_medio( numerador = kwargs['cli_pedi_total_pacientes_dia'],
                                        denominador = kwargs['cli_pedi_total_saidas'])
        res_cli_ad = kpi_tempo_medio( numerador = kwargs['cli_ad_total_pacientes_dia'],
                                        denominador = kwargs['cli_ad_total_saidas'])
        res_cli_idoso = kpi_tempo_medio( numerador = kwargs['cli_idoso_total_pacientes_dia'],
                                            denominador = kwargs['cli_idoso_total_saidas'])
        res_cir_pedi = kpi_tempo_medio( numerador = kwargs['cir_pedi_total_pacientes_dia'],
                                            denominador = kwargs['cir_pedi_total_saidas'])
        res_cir_ad = kpi_tempo_medio( numerador = kwargs['cir_ad_total_pacientes_dia'],
                                        denominador = kwargs['cir_ad_total_saidas'])
        res_cir_idoso = kpi_tempo_medio( numerador = kwargs['cir_idoso_total_pacientes_dia'],
                                            denominador = kwargs['cir_idoso_total_saidas'])
        res_clinico = kpi_tempo_medio( numerador = cli_total_pacientes_dia,
                                        denominador = cli_total_saidas)
        res_cirurgico = kpi_tempo_medio( numerador = cir_total_pacientes_dia,
                                            denominador = cir_total_saidas)
        res_pedi = kpi_tempo_medio( numerador = pedi_total_pacientes_dia,
                                        denominador = pedi_total_saidas)
        res_ad = kpi_tempo_medio( numerador = ad_total_pacientes_dia,
                                    denominador = ad_total_saidas)
        res_idoso = kpi_tempo_medio( numerador = idoso_total_pacientes_dia,
                                        denominador = idoso_total_saidas)
        res_geral = kpi_tempo_medio( numerador = total_pacientes_dia,
                                        denominador = total_saidas)
        return {name: value for name, value in locals().items() if name.startswith('res_')}
    else:
        return None

In [ ]:
resultados = kpi5( cli_pedi_total_pacientes_dia = df.at[0,'cli_pedi_pacientes_dia'],
      cli_pedi_total_saidas = df.at[0,'cli_pedi_saidas'], 
      cli_ad_total_pacientes_dia = df.at[0,'cli_ad_pacientes_dia'],
      cli_ad_total_saidas = df.at[0,'cli_ad_saidas'],
      cli_idoso_total_pacientes_dia = df.at[0,'cli_idoso_pacientes_dia'],
      cli_idoso_total_saidas = df.at[0,'cli_idoso_saidas'],
      cir_pedi_total_pacientes_dia = df.at[0,'cir_pedi_pacientes_dia'],
      cir_pedi_total_saidas = df.at[0,'cir_pedi_saidas'],
      cir_ad_total_pacientes_dia = df.at[0,'cir_ad_pacientes_dia'],
      cir_ad_total_saidas = df.at[0,'cir_ad_saidas'],
      cir_idoso_total_pacientes_dia = df.at[0,'cir_idoso_pacientes_dia'],
      cir_idoso_total_saidas = df.at[0,'cir_idoso_saidas'],
     )

In [ ]:
def kpi_taxa(numerador,denominador):
    return (numerador/denominador)[0] *100

def kpi1(**kwargs):
    # 1. Proporção de partos vaginais 

    # Validar se kwargs está vazio
    if not kwargs:
        raise ValueError("Os argumentos kwargs não podem estar vazios.")

    # Validar se existem os argumentos necessários
    if 'total_partos_vaginais' not in kwargs or 'total_partos_cesareos' not in kwargs:
        raise KeyError("Os valores de 'total_partos_vaginais' e 'total_partos_cesareos' são obrigatórios.")

    # Validar se os valores não são nulos
    if kwargs.get('total_partos_vaginais') is None or kwargs.get('total_partos_cesareos') is None:
        raise ValueError("Os valores de 'total_partos_vaginais' e 'total_partos_cesareos' não podem ser nulos.")
    
    # Validar se o número de argumentos é correto
    if len(kwargs) != 2:
        raise Exception(f"KPI1 requer 2 elementos. Foram passados {len(kwargs)}.")
    
    # Calcular o KPI
    total_partos = kwargs['total_partos_vaginais'] + kwargs['total_partos_cesareos']
    
    return kpi_taxa( numerador = kwargs['total_partos_vaginais'],
                            denominador = total_partos )

In [ ]:
# 1. Proporção de partos vaginais   
kpi1 = ( df.partos_vaginais / (df.partos_vaginais + df.partos_cesareos) )[0] *100

# 2. Proporção de reinternações em até 30 dias da saída hospitalar
kpi2_cli = ( df.reinternacoes_clinicas / df.saidas_clinicas_anterior )[0] *100
kpi2_cir = ( df.reinternacoes_cirurgicas / df.saidas_cirurgicas_anterior )[0] *100
kpi2 = ((df.reinternacoes_clinicas+df.reinternacoes_cirurgicas) / (df.saidas_clinicas_anterior+df.saidas_cirurgicas_anterior))[0]* 100

# 3. Taxa de parada cardiorrespiratória em unidade de internação
kpi3 = ( df.pcr_eventos / df.pacientes_dia )[0] *1000

# 4. Taxa de mortalidade institucional
kpi4_cli_neo_precoce = ( df.cli_neo_precoce_obitos / df.cli_neo_precoce_saidas )[0] *100
kpi4_cli_neo_tardio  = ( df.cli_neo_tardio_obitos  / df.cli_neo_tardio_saidas  )[0] *100
kpi4_cli_pedi        = ( df.cli_pedi_obitos        / df.cli_pedi_saidas        )[0] *100
kpi4_cli_adulto      = ( df.cli_ad_obitos          / df.cli_ad_saidas          )[0] *100
kpi4_cli_idoso       = ( df.cli_idoso_obitos       / df.cli_idoso_saidas       )[0] *100

kpi4_cir_neo_precoce = ( df.cir_neo_precoce_obitos / df.cir_neo_precoce_saidas )[0] *100
kpi4_cir_neo_tardio  = ( df.cir_neo_tardio_obitos  / df.cir_neo_tardio_saidas  )[0] *100
kpi4_cir_pedi        = ( df.cir_pedi_obitos        / df.cir_pedi_saidas        )[0] *100
kpi4_cir_adulto      = ( df.cir_ad_obitos          / df.cir_ad_saidas          )[0] *100
kpi4_cir_idoso       = ( df.cir_idoso_obitos       / df.cir_idoso_saidas       )[0] *100

kpi4_cli = (
    ( df.cli_neo_precoce_obitos+
        df.cli_neo_tardio_obitos+
        df.cli_pedi_obitos+
        df.cli_ad_obitos+
        df.cli_idoso_obitos ) / 
    ( df.cli_neo_precoce_saidas+
        df.cli_neo_tardio_saidas+
        df.cli_pedi_saidas+
        df.cli_ad_saidas+
        df.cli_idoso_saidas )
)[0] *100

kpi4_cir = (
    ( df.cir_neo_precoce_obitos+
        df.cir_neo_tardio_obitos+
        df.cir_pedi_obitos+
        df.cir_ad_obitos+
        df.cir_idoso_obitos ) / 
    ( df.cir_neo_precoce_saidas+
        df.cir_neo_tardio_saidas+
        df.cir_pedi_saidas+
        df.cir_ad_saidas+
        df.cir_idoso_saidas )
)[0] *100

kpi4_neo_precoce = (
    ( df.cli_neo_precoce_obitos+df.cir_neo_precoce_obitos ) / 
    ( df.cli_neo_precoce_saidas+df.cir_neo_precoce_saidas )
)[0] *100

kpi4_neo_tardio = (
    ( df.cli_neo_tardio_obitos+df.cir_neo_tardio_obitos ) / 
    ( df.cli_neo_tardio_saidas+df.cir_neo_tardio_saidas )
)[0] *100

kpi4_pedi = (
    ( df.cli_pedi_obitos+df.cir_pedi_obitos ) / 
    ( df.cli_pedi_saidas+df.cir_pedi_saidas )
)[0] *100

kpi4_ad = (
    ( df.cli_ad_obitos+df.cir_ad_obitos ) / 
    ( df.cli_ad_saidas+df.cir_ad_saidas )
)[0] *100

kpi4_idoso = (
    ( df.cli_idoso_obitos+df.cir_idoso_obitos ) / 
    ( df.cli_idoso_saidas+df.cir_idoso_saidas )
)[0] *100

kpi4_cir = (
    ( df.cir_neo_precoce_obitos+
        df.cir_neo_tardio_obitos+
        df.cir_pedi_obitos+
        df.cir_ad_obitos+
        df.cir_idoso_obitos ) / 
    ( df.cir_neo_precoce_saidas+
        df.cir_neo_tardio_saidas+
        df.cir_pedi_saidas+
        df.cir_ad_saidas+
        df.cir_idoso_saidas )
)[0] *100    

kpi4 = ((
    df.cli_neo_precoce_obitos+
    df.cli_neo_tardio_obitos+
    df.cli_pedi_obitos+
    df.cli_ad_obitos+
    df.cli_idoso_obitos+
    df.cir_neo_precoce_obitos+
    df.cir_neo_tardio_obitos+
    df.cir_pedi_obitos+
    df.cir_ad_obitos+
    df.cir_idoso_obitos
) / (
    df.cli_neo_precoce_saidas+
    df.cli_neo_tardio_saidas+
    df.cli_pedi_saidas+
    df.cli_ad_saidas+
    df.cli_idoso_saidas+
    df.cir_neo_precoce_saidas+
    df.cir_neo_tardio_saidas+
    df.cir_pedi_saidas+
    df.cir_ad_saidas+
    df.cir_idoso_saidas
))[0] *100

# 5. Tempo médio de internação
kpi5_cli_pedi   = ( df.cli_pedi_pacientes_dia  / df.cli_pedi_saidas)[0]
kpi5_cli_adulto = ( df.cli_ad_pacientes_dia    / df.cli_ad_saidas)[0]
kpi5_cli_idoso  = ( df.cli_idoso_pacientes_dia / df.cli_idoso_saidas)[0]
kpi5_cir_pedi   = ( df.cir_pedi_pacientes_dia  / df.cir_pedi_saidas)[0]
kpi5_cir_adulto = ( df.cir_ad_pacientes_dia    / df.cir_ad_saidas)[0]
kpi5_cir_idoso  = ( df.cir_idoso_pacientes_dia / df.cir_idoso_saidas)[0]
kpi5_cli = ( ( df.cli_pedi_pacientes_dia+df.cli_ad_pacientes_dia+df.cli_idoso_pacientes_dia ) / 
                ( df.cli_pedi_saidas+df.cli_ad_saidas+df.cli_idoso_saidas ) )[0]
kpi5_cir = ( ( df.cir_pedi_pacientes_dia+df.cir_ad_pacientes_dia+df.cir_idoso_pacientes_dia ) / 
                ( df.cir_pedi_saidas+df.cir_ad_saidas+df.cir_idoso_saidas ) )[0]
kpi5_pedi = ( ( df.cli_pedi_pacientes_dia+df.cir_pedi_pacientes_dia ) / 
                ( df.cli_pedi_saidas+df.cir_pedi_saidas ) )[0] *100
kpi5_ad = ( ( df.cli_ad_pacientes_dia+df.cir_ad_pacientes_dia ) / 
            ( df.cli_ad_saidas+df.cir_ad_saidas ) )[0]
kpi5_idoso = ( ( df.cli_idoso_pacientes_dia+df.cir_idoso_pacientes_dia ) / 
                ( df.cli_idoso_saidas+df.cir_idoso_saidas ) )[0]
kpi5 =  (( df.cli_pedi_pacientes_dia+df.cli_ad_pacientes_dia+df.cli_idoso_pacientes_dia+
            df.cir_pedi_pacientes_dia+df.cir_ad_pacientes_dia+df.cir_idoso_pacientes_dia ) / (
            df.cli_pedi_saidas+df.cli_ad_saidas+df.cli_idoso_saidas+
            df.cir_pedi_saidas+df.cir_ad_saidas+df.cir_idoso_saidas ))[0]

# 6. Tempo médio de permanência na emergência
kpi6 = ( df.total_tempo_permanencia_emergencia_hr / df.total_pacientes_emergencia )[0]

# 7. Tempo médio de espera na emergência para primeiro atendimento
kpi7_nvl2 = ( df.tempo_total_emergencia_nivel2_min / df.pacientes_emergencia_nivel2 )[0]
kpi7_nvl3 = ( df.tempo_total_emergencia_nivel3_min / df.pacientes_emergencia_nivel3 )[0]
kpi7 = ( ( df.tempo_total_emergencia_nivel2_min+df.tempo_total_emergencia_nivel3_min ) /
            ( df.pacientes_emergencia_nivel2+df.pacientes_emergencia_nivel3 ) )[0]

# 8. Taxa de início de antibiótico intravenoso profilático
kpi8 = ( df.cirurgias_com_antibiotico / df.total_cirurgias_limpas )[0] *100

# 9. Taxa de infecção de sítio cirúrgico em cirurgia limpa
kpi9 = ( df.total_infeccoes / df.total_cirurgias_limpas )[0] *100

# 10. Densidade de incidência de infecção primária de corrente sanguínea (IPCS)
# em pacientes em uso de cateter venoso central (CVC)
kpi10_neo = ( ( df.unid_int_neo_precoce_infec + df.unid_int_neo_tardio_infec ) / 
                ( df.unid_int_neo_precoce_cvc_dia + df.unid_int_neo_tardio_cvc_dia ) )[0] *1000
kpi10_pedi = ( df.unid_int_pedi_infec / df.unid_int_pedi_cvc_dia )[0] *1000
kpi10_ad = ( ( df.unid_int_ad_infec + df.unid_int_idoso_infec ) /
                ( df.unid_int_ad_cvc_dia + df.unid_int_idoso_cvc_dia ) )[0] *1000
kpi10_neo_uti = ( ( df.uti_neo_precoce_infec + df.uti_neo_tardio_infec ) / 
                    ( df.uti_neo_precoce_cvc_dia + df.uti_neo_tardio_cvc_dia ) )[0] *1000
kpi10_pedi_uti = ( df.uti_pedi_infec / df.uti_pedi_cvc_dia )[0] *1000
kpi10_ad_uti = ( ( df.uti_ad_infec + df.uti_idoso_infec ) / 
                    ( df.uti_ad_cvc_dia + df.uti_idoso_cvc_dia ) )[0] *1000
kpi10_unid_int = (
    (   df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec+
        df.unid_int_pedi_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec
    ) /                 
    (   df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia+
        df.unid_int_pedi_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia
    )
)[0] *1000
kpi10_uti = (
    (   df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.uti_pedi_infec+
        df.uti_ad_infec+
        df.uti_idoso_infec
    ) /                 
    (   df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.uti_pedi_cvc_dia+
        df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia
    )
)[0] *1000      
kpi10_neo = (
    (   df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec
    ) /                 
    (   df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia
    )
)[0] *1000
kpi10_pedi = (
    (   df.uti_pedi_infec+
        df.unid_int_pedi_infec
    ) /                 
    (   df.uti_pedi_cvc_dia+
        df.unid_int_pedi_cvc_dia
    )
)[0] *1000            
kpi10_ad = (
    (   df.uti_ad_infec+
        df.uti_idoso_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec
    ) /                 
    (   df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia
    )
)[0] *1000

kpi10 = (
    (   df.unid_int_neo_precoce_infec+
        df.unid_int_neo_tardio_infec+
        df.unid_int_pedi_infec+
        df.unid_int_ad_infec+
        df.unid_int_idoso_infec+
        df.uti_neo_precoce_infec+
        df.uti_neo_tardio_infec+
        df.uti_pedi_infec+
        df.uti_ad_infec+
        df.uti_idoso_infec
    ) /                 
    (   df.unid_int_neo_precoce_cvc_dia+
        df.unid_int_neo_tardio_cvc_dia+
        df.unid_int_pedi_cvc_dia+
        df.unid_int_ad_cvc_dia+
        df.unid_int_idoso_cvc_dia+
        df.uti_neo_precoce_cvc_dia+
        df.uti_neo_tardio_cvc_dia+
        df.uti_pedi_cvc_dia+
        df.uti_ad_cvc_dia+
        df.uti_idoso_cvc_dia
    )
)[0] *1000

# 11. Densidade de incidência de infecção do trato urinário (ITU) associada a um
# cateter vesical de demora (CVD)
kpi11_neo = ( ( df.unid_int_neo_precoce_itu + df.unid_int_neo_tardio_itu ) / 
                ( df.unid_int_neo_precoce_cvd_dia + df.unid_int_neo_tardio_cvd_dia ) )[0] *1000
kpi11_pedi = ( df.unid_int_pedi_itu / df.unid_int_pedi_cvd_dia )[0] *1000
kpi11_ad = ( ( df.unid_int_ad_itu + df.unid_int_idoso_itu ) /
                ( df.unid_int_ad_cvd_dia + df.unid_int_idoso_cvd_dia ) )[0] *1000
kpi11_neo_uti = ( ( df.uti_neo_precoce_itu + df.uti_neo_tardio_itu ) / 
                    ( df.uti_neo_precoce_cvd_dia + df.uti_neo_tardio_cvd_dia ) )[0] *1000
kpi11_pedi_uti = ( df.uti_pedi_itu / df.uti_pedi_cvd_dia )[0] *1000
kpi11_ad_uti = ( ( df.uti_ad_itu + df.uti_idoso_itu ) / 
                    ( df.uti_ad_cvd_dia + df.uti_idoso_cvd_dia ) )[0] *1000
kpi11_unid_int = (
    (   df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu+
        df.unid_int_pedi_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu
    ) /                 
    (   df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia+
        df.unid_int_pedi_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia
    )
)[0] *1000
kpi11_uti = (
    (   df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.uti_pedi_itu+
        df.uti_ad_itu+
        df.uti_idoso_itu
    ) /                 
    (   df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.uti_pedi_cvd_dia+
        df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia
    )
)[0] *1000      
kpi11_neo = (
    (   df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu
    ) /                 
    (   df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia
    )
)[0] *1000
kpi11_pedi = (
    (   df.uti_pedi_itu+
        df.unid_int_pedi_itu
    ) /                 
    (   df.uti_pedi_cvd_dia+
        df.unid_int_pedi_cvd_dia
    )
)[0] *1000            
kpi11_ad = (
    (   df.uti_ad_itu+
        df.uti_idoso_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu
    ) /                 
    (   df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia
    )
)[0] *1000 
kpi11 = (
    (   df.unid_int_neo_precoce_itu+
        df.unid_int_neo_tardio_itu+
        df.unid_int_pedi_itu+
        df.unid_int_ad_itu+
        df.unid_int_idoso_itu+
        df.uti_neo_precoce_itu+
        df.uti_neo_tardio_itu+
        df.uti_pedi_itu+
        df.uti_ad_itu+
        df.uti_idoso_itu
    ) /                 
    (   df.unid_int_neo_precoce_cvd_dia+
        df.unid_int_neo_tardio_cvd_dia+
        df.unid_int_pedi_cvd_dia+
        df.unid_int_ad_cvd_dia+
        df.unid_int_idoso_cvd_dia+
        df.uti_neo_precoce_cvd_dia+
        df.uti_neo_tardio_cvd_dia+
        df.uti_pedi_cvd_dia+
        df.uti_ad_cvd_dia+
        df.uti_idoso_cvd_dia
    )
)[0] *1000

# 12. Taxa de profilaxia de tromboembolismo venoso
kpi12_cli          = ( df.cli_profilaxia          / df.cli_total_pacientes )[0] *100
kpi12_cir_orto     = ( df.cir_orto_profilaxia     / df.cir_orto_total_pacientes )[0] *100
kpi12_cir_nao_orto = ( df.cir_nao_orto_profilaxia / df.cir_nao_orto_total_pacientes )[0] *100
kpi12 = ( ( df.cli_profilaxia+df.cir_orto_profilaxia+df.cir_nao_orto_profilaxia ) /
            ( df.cli_total_pacientes+df.cir_orto_total_pacientes+df.cir_nao_orto_total_pacientes )
        )[0] *100

# 13. Densidade de incidência de queda resultando em lesão em paciente
kpi13 = ( df.quedas_com_dano / 
            ( df.cli_neo_precoce_pacientes_dia+
            df.cli_neo_tardio_pacientes_dia+
            df.cli_pedi_pacientes_dia+
            df.cli_ad_pacientes_dia+
            df.cli_idoso_pacientes_dia+
            df.cir_neo_precoce_pacientes_dia+
            df.cir_neo_tardio_pacientes_dia+
            df.cir_pedi_pacientes_dia+
            df.cir_ad_pacientes_dia+
            df.cir_idoso_pacientes_dia )
            )[0] *1000

# 14. Evento sentinela
kpi14 = ( df.eventos_sentinela / 
            ( df.cli_neo_precoce_pacientes_dia+
            df.cli_neo_tardio_pacientes_dia+
            df.cli_pedi_pacientes_dia+
            df.cli_ad_pacientes_dia+
            df.cli_idoso_pacientes_dia+
            df.cir_neo_precoce_pacientes_dia+
            df.cir_neo_tardio_pacientes_dia+
            df.cir_pedi_pacientes_dia+
            df.cir_ad_pacientes_dia+
            df.cir_idoso_pacientes_dia )
        )[0] *1000

In [ ]:
kpis = [kpi1, 
kpi2_cli, kpi2_cir, kpi2,
kpi3,
kpi4_cli_neo_precoce, kpi4_cli_neo_tardio, kpi4_cli_pedi, kpi4_cli_adulto, kpi4_cli_idoso, kpi4_cli,
kpi4_cir_neo_precoce, kpi4_cir_neo_tardio, kpi4_cir_pedi, kpi4_cir_adulto, kpi4_cir_idoso, kpi4_cir,
kpi4_neo_precoce, kpi4_neo_tardio, kpi4_pedi, kpi4_ad, kpi4_idoso, kpi4,
kpi5_cli_pedi, kpi5_cli_adulto, kpi5_cli_idoso, kpi5_cli,
kpi5_cir_pedi, kpi5_cir_adulto, kpi5_cir_idoso, kpi5_cir,
kpi5_pedi, kpi5_ad, kpi5_idoso, kpi5,
kpi6,
kpi7_nvl2, kpi7_nvl3, kpi7,
kpi8,
kpi9,
kpi10_neo, kpi10_pedi, kpi10_ad, kpi10_neo_uti, kpi10_pedi_uti, kpi10_ad_uti, kpi10_unid_int, kpi10_uti, kpi10,
kpi11_neo, kpi11_pedi, kpi11_ad, kpi11_neo_uti, kpi11_pedi_uti, kpi11_ad_uti, kpi11_unid_int, kpi11_uti, kpi11,
kpi12_cli, kpi12_cir_orto, kpi12_cir_nao_orto, kpi12,
kpi13,
kpi14]

#### prd

In [1]:
from KPI           import KPI
from dbConfig      import dbConfig
from bson.objectid import ObjectId
import pandas  as pd
import pymongo as db
import config

dbC = dbConfig()
kpi = KPI()

In [2]:
def make_dataframe(data:dict) -> pd.DataFrame:
    if (not data):
        raise Exception("Collection data not found.")
    else:
        try:
            return pd.json_normalize(data)
        except ValueError as e:
            raise Exception("Unable to load data from the collection: ", e)

In [3]:
metricas = dbC.get_data( collection_name="metrics",
                        query= { "organization_id": ObjectId("6745d7873f4e39e161319575"),
                                    "year": 2023,
                                    "month": 2 } )

df = make_dataframe( data=metricas )
#df.info()

In [5]:
_1_proporcao_partos_vaginais = kpi.kpi1( total_partos_vaginais = df.at[0,'partos_vaginais'],
                                            total_partos_cesareos = df.at[0,'partos_cesareos'] )
_2_proporcao_reinternacoes_30_dias = kpi.kpi2 ( cli_total_reinternacoes_30_dias = df.at[0,'reinternacoes_clinicas'],
                                                cli_total_saida_mes_anterior = df.at[0,'saidas_clinicas_anterior'],
                                                cir_total_reinternacoes_30_dias = df.at[0,'reinternacoes_cirurgicas'],
                                                cir_total_saida_mes_anterior = df.at[0,'saidas_cirurgicas_anterior'] )
_3_taxa_pcr = kpi.kpi3( total_pcr = df.at[0,'pcr_eventos'],
                        total_pacientes_dia = df.at[0,'pacientes_dia'] )
_4_taxa_mortalidade = kpi.kpi4( cli_neo_precoce_total_obitos = df.at[0,'cli_neo_precoce_obitos'],
                                cli_neo_precoce_total_saidas = df.at[0,'cli_neo_precoce_saidas'],
                                cli_neo_tardio_total_obitos = df.at[0,'cli_neo_tardio_obitos'],
                                cli_neo_tardio_total_saidas = df.at[0,'cli_neo_tardio_saidas'],
                                cli_pedi_total_obitos = df.at[0,'cli_pedi_obitos'],
                                cli_pedi_total_saidas = df.at[0,'cli_pedi_saidas'],
                                cli_ad_total_obitos = df.at[0,'cli_ad_obitos'],
                                cli_ad_total_saidas = df.at[0,'cli_ad_saidas'],
                                cli_idoso_total_obitos = df.at[0,'cli_idoso_obitos'],
                                cli_idoso_total_saidas = df.at[0,'cli_idoso_saidas'],
                                cir_neo_precoce_total_obitos = df.at[0,'cir_neo_precoce_obitos'],
                                cir_neo_precoce_total_saidas = df.at[0,'cir_neo_precoce_saidas'],
                                cir_neo_tardio_total_obitos = df.at[0,'cir_neo_tardio_obitos'],
                                cir_neo_tardio_total_saidas = df.at[0,'cir_neo_tardio_saidas'],
                                cir_pedi_total_obitos = df.at[0,'cir_pedi_obitos'],
                                cir_pedi_total_saidas = df.at[0,'cir_pedi_saidas'],
                                cir_ad_total_obitos = df.at[0,'cir_ad_obitos'],
                                cir_ad_total_saidas = df.at[0,'cir_ad_saidas'],
                                cir_idoso_total_obitos = df.at[0,'cir_idoso_obitos'],
                                cir_idoso_total_saidas = df.at[0,'cir_idoso_saidas'] )
_5_tempo_medio_internacao = kpi.kpi5( cli_pedi_total_pacientes_dia = df.at[0,'cli_pedi_pacientes_dia'],
                                      cli_pedi_total_saidas = df.at[0,'cli_pedi_saidas'],
                                      cli_ad_total_pacientes_dia = df.at[0,'cli_ad_pacientes_dia'],
                                      cli_ad_total_saidas = df.at[0,'cli_ad_saidas'],
                                      cli_idoso_total_pacientes_dia = df.at[0,'cli_idoso_pacientes_dia'],
                                      cli_idoso_total_saidas = df.at[0,'cli_idoso_saidas'],
                                      cir_pedi_total_pacientes_dia = df.at[0,'cir_pedi_pacientes_dia'],
                                      cir_pedi_total_saidas = df.at[0,'cir_pedi_saidas'],
                                      cir_ad_total_pacientes_dia = df.at[0,'cir_ad_pacientes_dia'],
                                      cir_ad_total_saidas = df.at[0,'cir_ad_saidas'],
                                      cir_idoso_total_pacientes_dia = df.at[0,'cir_idoso_pacientes_dia'],
                                      cir_idoso_total_saidas = df.at[0,'cir_idoso_saidas'] )
_6_tempo_medio_emergencia = kpi.kpi6( total_tempo_entrada_termino = df.at[0,'total_tempo_permanencia_emergencia_hr'],
                                      total_pacientes_buscaram_atendimento = df.at[0,'total_pacientes_emergencia'] )
_7_tempo_medio_espera_emergencia = kpi.kpi7( nvl2_total_tempo_espera = df.at[0,'tempo_total_emergencia_nivel2_min'],
                                             nvl2_total_pacientes_buscaram_atendimento = df.at[0,'pacientes_emergencia_nivel2'],
                                             nvl3_total_tempo_espera = df.at[0,'tempo_total_emergencia_nivel3_min'],
                                             nvl3_total_pacientes_buscaram_atendimento = df.at[0,'pacientes_emergencia_nivel3'] )
_8_taxa_atb_profilatico = kpi.kpi8( total_cirurgias_limpas_com_atb = df.at[0,'cirurgias_com_antibiotico'],
                                    total_cirurgias_limpas = df.at[0,'total_cirurgias_limpas'] )
_9_taxa_infeccao_cirurgia_limpa = kpi.kpi9( total_isc_30_dias = df.at[0,'total_infeccoes'],
                                            total_cirurgias_limpas_mes_anterior = df.at[0,'total_cirurgias_limpas_anterior'] )
_10_incidencia_ipcs_cvc = kpi.kpi10( ui_neo_total_ipcs = df.at[0,'ui_neo_infec'],
                                     uti_neo_total_ipcs = df.at[0,'uti_neo_infec'],
                                     ui_pedi_total_ipcs = df.at[0,'ui_pedi_infec'],
                                     uti_pedi_total_ipcs = df.at[0,'uti_pedi_infec'],
                                     ui_ad_total_ipcs = df.at[0,'ui_ad_infec'],
                                     uti_ad_total_ipcs = df.at[0,'uti_ad_infec'],
                                     ui_neo_total_cvc_dia = df.at[0,'ui_neo_cvc_dia'],
                                     uti_neo_total_cvc_dia = df.at[0,'uti_neo_cvc_dia'],
                                     ui_pedi_total_cvc_dia = df.at[0,'ui_pedi_cvc_dia'],
                                     uti_pedi_total_cvc_dia = df.at[0,'uti_pedi_cvc_dia'],
                                     ui_ad_total_cvc_dia = df.at[0,'ui_ad_cvc_dia'],
                                     uti_ad_total_cvc_dia = df.at[0,'uti_ad_cvc_dia'] )
_11_incidencia_itu_cvd = kpi.kpi11( ui_neo_total_itu = df.at[0,'ui_neo_itu'],
                                    uti_neo_total_itu = df.at[0,'uti_neo_itu'],
                                    ui_pedi_total_itu = df.at[0,'ui_pedi_itu'],
                                    uti_pedi_total_itu = df.at[0,'uti_pedi_itu'],
                                    ui_ad_total_itu = df.at[0,'ui_ad_itu'],
                                    uti_ad_total_itu = df.at[0,'uti_ad_itu'],
                                    ui_neo_total_cvd_dia = df.at[0,'ui_neo_cvd_dia'],
                                    uti_neo_total_cvd_dia = df.at[0,'uti_neo_cvd_dia'],
                                    ui_pedi_total_cvd_dia = df.at[0,'ui_pedi_cvd_dia'],
                                    uti_pedi_total_cvd_dia = df.at[0,'uti_pedi_cvd_dia'],
                                    ui_ad_total_cvd_dia = df.at[0,'ui_ad_cvd_dia'],
                                    uti_ad_total_cvd_dia = df.at[0,'uti_ad_cvd_dia'] )
_12_taxa_profilaxia_tromboembolismo = kpi.kpi12( cli_total_pacientes_risco_profilaxia_TEV = df.at[0,'cli_profilaxia'],
                                                 cli_total_pacientes_risco = df.at[0,'cli_total_pacientes'],
                                                 cir_orto_total_pacientes_risco_profilaxia_TEV = df.at[0,'cir_orto_profilaxia'],
                                                 cir_orto_total_pacientes_risco = df.at[0,'cir_orto_total_pacientes'],
                                                 cir_n_orto_total_pacientes_risco_profilaxia_TEV = df.at[0,'cir_nao_orto_profilaxia'],
                                                 cir_n_orto_total_pacientes_risco = df.at[0,'cir_nao_orto_total_pacientes'] )
_13_incidencia_queda = kpi.kpi13( total_quedas_dano = df.at[0,'quedas_com_dano'],
                                  total_pacientes_dia = df.at[0,'pacientes_dia'] )
_14_evento_sentinela = kpi.kpi14( total_eventos_sentinela = df.at[0,'eventos_sentinela'],
                                  total_pacientes_dia = df.at[0,'pacientes_dia'] )


In [16]:
query = {
    "organization_id": ObjectId("6745d7873f4e39e161319575"),
    "year": 2023,
    "month": 1,
    "rkpi1": _1_proporcao_partos_vaginais,
    **_2_proporcao_reinternacoes_30_dias,
    "rkpi3": _3_taxa_pcr,
    **_4_taxa_mortalidade,
    **_5_tempo_medio_internacao,
    "rkpi6": _6_tempo_medio_emergencia,
    **_7_tempo_medio_espera_emergencia,
    "rkpi8": _8_taxa_atb_profilatico,
    "rkpi9": _9_taxa_infeccao_cirurgia_limpa,
    **_10_incidencia_ipcs_cvc,
    **_11_incidencia_itu_cvd,
    **_12_taxa_profilaxia_tromboembolismo,
    "rkpi13": _13_incidencia_queda,
    "rkpi14": _14_evento_sentinela
}

In [17]:
query.keys()

dict_keys(['organization_id', 'year', 'month', 'rkpi1', 'rkpi2_clinico', 'rkpi2_cirurgico', 'rkpi2_geral', 'rkpi3', 'rkpi4_cli_neo_precoce', 'rkpi4_cli_neo_tardio', 'rkpi4_cli_pedi', 'rkpi4_cli_ad', 'rkpi4_cli_idoso', 'rkpi4_cir_neo_precoce', 'rkpi4_cir_neo_tardio', 'rkpi4_cir_pedi', 'rkpi4_cir_ad', 'rkpi4_cir_idoso', 'rkpi4_clinico', 'rkpi4_cirurgico', 'rkpi4_neo_precoce', 'rkpi4_neo_tardio', 'rkpi4_pedi', 'rkpi4_ad', 'rkpi4_idoso', 'rkpi4_geral', 'rkpi5_cli_pedi', 'rkpi5_cli_ad', 'rkpi5_cli_idoso', 'rkpi5_cir_pedi', 'rkpi5_cir_ad', 'rkpi5_cir_idoso', 'rkpi5_clinico', 'rkpi5_cirurgico', 'rkpi5_pedi', 'rkpi5_ad', 'rkpi5_idoso', 'rkpi5_geral', 'rkpi6', 'rkpi7_nvl2', 'rkpi7_nvl3', 'rkpi7_geral', 'rkpi8', 'rkpi9', 'rkpi10_ui_neo', 'rkpi10_ui_pedi', 'rkpi10_ui_ad', 'rkpi10_uti_neo', 'rkpi10_uti_pedi', 'rkpi10_uti_ad', 'rkpi10_neo', 'rkpi10_pedi', 'rkpi10_ad', 'rkpi10_ui', 'rkpi10_uti', 'rkpi10_geral', 'rkpi11_ui_neo', 'rkpi11_ui_pedi', 'rkpi11_ui_ad', 'rkpi11_uti_neo', 'rkpi11_uti_pedi', '

In [7]:
try:
    client = db.MongoClient(config.MONGO_URI)
    database = client.get_database(config.DATABASE)
    collection = database.get_collection("kpi_results")
    metrics = collection.insert_one(query)
    client.close()
except Exception as e:
    raise Exception("Unable to retrieve the document due to the following error: ", e)

In [10]:
metrics.acknowledged

True